In [6]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.stats.outliers_influence import reset_ramsey
# import seaborn as sns
from scipy import stats
import re
# import scipy.stats as stats

In [7]:
source_file = 'Labour.csv'
url = 'https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/'
df = pd.read_csv(url + source_file)
df

,capital,labour,output,wage
0,2.606563,184,9.250759,43.080307
1,1.323237,91,3.664310,27.780016
2,22.093692,426,28.781516,44.467748
3,10.737851,72,4.124642,39.734710
4,1.161365,46,2.890150,34.650709
...,...,...,...,...
564,2.625403,20,1.424376,33.477545
565,1.276386,61,2.109048,26.300732
566,1.953869,117,6.241870,41.153979
567,1.318527,46,7.902237,66.720139


### 1.4 output equation #2

Для набора данных `Labour` рассмотрим линейную регрессию **log(output)** на **log(capital), log(labour), log(wage)**.

In [8]:
formula = (
    f'np.log(output) ~ '
    f'np.log(capital) + np.log(labour) + np.log(wage)'
)
model = smf.ols(formula = formula, data = df).fit()
# formula = 'sleep ~ totwrk + age + smsa + male + totwrk:male+age:male+smsa:male'
# Sleep_2 = smf.ols(formula = formula, data = df).fit()
# info_dict = {
#              'R-squared': lambda x: "%#8.3f" % x.rsquared,
#              }
print(
    summary_col(
        model,
        float_format='%.3f',
        stars=True,
        # info_dict = info_dict
    )
)



                np.log(output)
------------------------------
Intercept       -5.007***     
                (0.221)       
np.log(capital) 0.149***      
                (0.015)       
np.log(labour)  0.720***      
                (0.019)       
np.log(wage)    0.921***      
                (0.058)       
R-squared       0.888         
R-squared Adj.  0.888         
Standard errors in
parentheses.
* p<.1, ** p<.05, ***p<.01


In [9]:
# Выполняем RESET тест
reset = reset_ramsey(model, degree=4)
df1 = reset.df_num
df2 = reset.df_denom
f_obs = reset.fvalue

print(f'RESET = {f_obs:.3f} df1 = {df1:.0f} df2 = {df2:.0f}')


RESET = 10.595 df1 = 3 df2 = 562


In [10]:
alpha = 0.05
f_cr = stats.f.ppf(1-alpha,3,len(df)-9).round(2)
print(f'F_cr = {f_cr:.2f}')

F_cr = 2.62


In [11]:
positive = 'Гипотеза отвергается, регрессия не согласуется с данными'
negative = 'Гипотеза не отвергается, регрессия согласуется с данными'
reply = positive if f_obs > f_cr else negative
print(f'{reply}')

Гипотеза отвергается, регрессия не согласуется с данными
